In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [20]:
tf.reset_default_graph()

In [21]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def sample_z(n,m):
    return np.random.uniform(-1.,1., size=[n,m])

In [22]:
# Defining input placeholder and parameters for the generator

Z = tf.placeholder(tf.float32, shape=[None, 100])
with tf.name_scope("generator"):
    G_W1 = tf.Variable(xavier_init([100, 128]),name="G_W1")
    G_b1 = tf.Variable(tf.zeros(shape=[128]),name="G_B1")

    G_W2 = tf.Variable(xavier_init([128, 784]),name="G_W2")
    G_b2 = tf.Variable(tf.zeros(shape=[784]),name="G_B2")

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [23]:
# Defining input placeholder and parameters for the discriminator

X = tf.placeholder(tf.float32, shape=[None, 784])

with tf.name_scope("discriminator"):

    D_W1 = tf.Variable(xavier_init([784, 128]), name="D_W1")
    D_b1 = tf.Variable(tf.zeros(shape=[128]),name="D_B1")

    D_W2 = tf.Variable(xavier_init([128, 1]),name="D_W2")
    D_b2 = tf.Variable(tf.zeros(shape=[1]),name="D_B2")

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [24]:
def generator(z):
    with tf.name_scope("generator"):
        G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
        G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
        G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [25]:
def discriminator(x):
    with tf.name_scope("discriminator"):
        D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
        D_out = tf.matmul(D_h1, D_W2) + D_b2
        #D_prob = tf.nn.sigmoid(D_log_prob)
    return D_out

In [26]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [27]:
G_sample = generator(Z)
D_real = discriminator(X)
D_fake = discriminator(G_sample)

In [28]:
D_loss = tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)
G_loss = -tf.reduce_mean(D_fake)

In [29]:
#D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list = theta_D)
#G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list = theta_G)

D_solver = (tf.train.RMSPropOptimizer(learning_rate=5e-5)
            .minimize(-D_loss, var_list=theta_D))
G_solver = (tf.train.RMSPropOptimizer(learning_rate=5e-5)
            .minimize(G_loss, var_list=theta_G))

clip_D = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in theta_D]

In [30]:
z_size = 100
mb_size = 16

mnist = input_data.read_data_sets('../data/MNIST_data', one_hot=True)

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [31]:
sess = tf.Session()
#writer = tf.summary.FileWriter('summaries')
#writer.add_graph(sess.graph)
sess.run(tf.initialize_all_variables())
#merge_summary=tf.summary.merge_all()

In [32]:
if not os.path.exists('wgan_out/'):
    os.makedirs('wgan_out/')

i = 0

for it in range(1000000):
    if it % 100 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_z(16, z_size)})

        fig = plot(samples)
        plt.savefig('wgan_out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)
    
    for _ in range(5):

        X_mb, _ = mnist.train.next_batch(mb_size)

        _, D_loss_curr, _ = sess.run([D_solver, D_loss, clip_D], feed_dict={X: X_mb, Z: sample_z(mb_size, z_size)})
        #tf.summary.scalar("D_loss", D_loss_curr)
    
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_z(mb_size, z_size)})

    #if it % 5 == 0:
        #s = sess.run(merge_summary, feed_dict={X:X_mb, Z: sample_z(mb_size, z_size)})
        #writer.add_summary(s,it)
    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: -0.002372
G_loss: -0.01272

Iter: 100
D loss: 1.622
G_loss: 1.246

Iter: 200
D loss: 1.657
G_loss: 1.229

Iter: 300
D loss: 1.621
G_loss: 1.114

Iter: 400
D loss: 1.455
G_loss: 0.9845

Iter: 500
D loss: 1.337
G_loss: 0.8694

Iter: 600
D loss: 1.299
G_loss: 0.7728

Iter: 700
D loss: 1.229
G_loss: 0.7285

Iter: 800
D loss: 0.9988
G_loss: 0.7699

Iter: 900
D loss: 0.9438
G_loss: 0.7084

Iter: 1000
D loss: 0.7688
G_loss: 0.5418

Iter: 1100
D loss: 0.6363
G_loss: 0.6228

Iter: 1200
D loss: 0.5666
G_loss: 0.6026

Iter: 1300
D loss: 0.4174
G_loss: 0.5446

Iter: 1400
D loss: 0.3702
G_loss: 0.4705

Iter: 1500
D loss: 0.3204
G_loss: 0.436

Iter: 1600
D loss: 0.2461
G_loss: 0.3841

Iter: 1700
D loss: 0.1642
G_loss: 0.3298

Iter: 1800
D loss: 0.1301
G_loss: 0.1897

Iter: 1900
D loss: 0.07655
G_loss: 0.06693

Iter: 2000
D loss: 0.05946
G_loss: 0.06208

Iter: 2100
D loss: 0.05941
G_loss: 0.05235

Iter: 2200
D loss: 0.03495
G_loss: 0.04227

Iter: 2300
D loss: 0.03788
G_loss: 0.01159



KeyboardInterrupt: 